In [1]:
import numpy as np
from osgeo import gdal




In [3]:


output = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\ftCollins_inputs_Feb182025_b.nc"

input1 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\dtm_FtCollins.tif"
input2 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\lat_grid.tif"
input3 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\lon_grid.tif"
input4 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\building_FtCollins_m_2025_b.tif"
#input5 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\landuse_FtCollins.tif"
input6 = r"C:\Users\boehnert\Documents\projects\FtCollins\FtCollins\final10km\tiff\nlcd_FtCollins.tif"

# Open the input TIFF files
input_raster1 = gdal.Open(input1)
input_raster2 = gdal.Open(input2)
input_raster3 = gdal.Open(input3)
input_raster4 = gdal.Open(input4)
#input_raster5 = gdal.Open(input5)
input_raster6 = gdal.Open(input6)


# Read the raster bands from both TIFF files
band1 = input_raster1.GetRasterBand(1)
band2 = input_raster2.GetRasterBand(1)
band3 = input_raster3.GetRasterBand(1)
band4 = input_raster4.GetRasterBand(1)
#band5 = input_raster5.GetRasterBand(1)
band6 = input_raster6.GetRasterBand(1)

# Read the data from the bands as NumPy arrays
data1 = band1.ReadAsArray()
data2 = band2.ReadAsArray()
data3 = band3.ReadAsArray()
data4 = band4.ReadAsArray()
#data5 = band5.ReadAsArray()
data6 = band6.ReadAsArray()

# Flip the data to correct the upside-down issue
data1 = np.flipud(data1)
data2 = np.flipud(data2)
data3 = np.flipud(data3)
data4 = np.flipud(data4)
data6 = np.flipud(data6)

# Get the spatial information from the first input file (assuming both files have the same dimensions)
geotransform = input_raster1.GetGeoTransform()
projection = input_raster1.GetProjection()

# Get the dimensions (rows and columns) of the input raster
rows = input_raster1.RasterYSize
cols = input_raster1.RasterXSize

# Create a NetCDF file
from netCDF4 import Dataset

# Create a new NetCDF file to store the data
ncfile = Dataset(output, "w", format="NETCDF4")

# Define the dimensions for the NetCDF file
ncfile.createDimension("x", cols)
ncfile.createDimension("y", rows)

# Create coordinate variables
x_var = ncfile.createVariable("x", np.float32, ("x",))
y_var = ncfile.createVariable("y", np.float32, ("y",))

# Compute coordinate values from the GeoTransform
x_values = np.linspace(geotransform[0], geotransform[0] + (cols - 1) * geotransform[1], cols)
y_values = np.linspace(geotransform[3], geotransform[3] + (rows - 1) * geotransform[5], rows)

# Assign coordinate values
x_var[:] = x_values
#y_var[:] = y_values
y_var[:] = np.flipud(y_values)

# Set units for clarity
x_var.units = "meters"
y_var.units = "meters"


# Create the variables to store the data arrays (one for each input TIFF)

var1 = ncfile.createVariable("topoPos", np.float32, ("y", "x"))
var2 = ncfile.createVariable("lat", np.float64, ("y", "x"))
var3 = ncfile.createVariable("lon", np.float64, ("y", "x"))
var4 = ncfile.createVariable("BuildingHeights", np.float32, ("y", "x"))
var5 = ncfile.createVariable("cellsize", np.float32)
var6 = ncfile.createVariable("LandCover", np.int32, ("y", "x"))

#data5.assign(1)
# Create a 2D array of 1s (default cell size)
data5 = np.array([[1]], dtype=np.float32)

# Assign this 2D array to the cellsize variable
#cellsize_var[:] = cellsize_data

# Write the data arrays into the variables
var1[:] = data1
var2[:] = data2
var3[:] = data3
var4[:] = data4
var5[:] = data5
var6[:] = data6

# Set the attributes for the variables (e.g., units, description)
var1.units = "meters"  # Set appropriate units
var2.units = "decimal degrees"  # Set appropriate units
var3.units =  "decimal degrees" # Set appropriate units
var4.units = "meters"  # Set appropriate units
var5.units = "meters"  # Set appropriate units
var6.units = "land cover cat"  # Set appropriate units

# Add metadata for the NetCDF file
ncfile.description = "NetCDF file created from input TIFFs"
ncfile.projection = projection
ncfile.geotransform = str(geotransform)

# Close the NetCDF file
ncfile.close()

print("NetCDF file created: output.nc")



NetCDF file created: output.nc
